In [1]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
scaler = StandardScaler()
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.preprocessing import kernel_normalization
from MKLpy.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics.pairwise import rbf_kernel as RBF
import pickle as pkl
###


from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 
from sklearn.metrics import roc_curve, auc

from MKLpy.metrics.pairwise.misc import homogeneous_polynomial_kernel as HPK_kernel
%matplotlib inline


In [2]:

def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file
def featureCreation(idxKey, locDict):
    ''' gives out clean features and labels for a given locDict and a idxKey '''
    keys=list(locDict.keys())
    featuresIdxDirFileLoc= locDict[keys[idxKey]][0]
    labelsIdxDirFileLoc= locDict[keys[idxKey]][1]
    ''' read the features file'''
    featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='latin1') 
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                 featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
    labelsDf=pd.read_csv(labelsIdxDirFileLoc)
    ''' pop the labels out'''
    labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[ labelName])
    arrX = np.array(dfX)
    ''' feature normalisation'''
    #feature scaling in [0,1] - X = rescale_01(arrX)
    X = normalization(rescale_01(arrX))
    y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
    ''' returns features, labels'''
    return X, y

In [7]:
hardDrivesLoc = '/media/ak/'
dataOnlyDrive = '/media/ak/DataOnly'  # external date only drive
ext_drive_loc = '/media/ak/My Passport/Experiment Data/'
data_dir = os.getenv('FINANCE_DATA')  # internal folder with finance data
#  input drive
inputDrive = hardDrivesLoc
''' Inputs '''
folderIdx = 0
folderList = [s for s in os.listdir(dataOnlyDrive) if s.startswith('Dat') or s.startswith('Fin')]
finalLocation = "/".join((dataOnlyDrive, folderList[folderIdx]))
symbols = [s for s in os.listdir(finalLocation) if s.endswith('.L')]  # keep a list of the symbols
print(symbols)
# picking up a specific symbol
symbolIdx = 2  # pick one of the symbols
# symbols[symbolIdx] -->output :PRU.L
# do a join to get the location
symbolLocation = "/".join((finalLocation, symbols[symbolIdx]))
# # get the features now
symbolFeaturesLocation = "/".join((symbolLocation, 'MODEL_BASED'))  # where all the HMM output is

# print(symbolFeaturesLocation) # <-- all the HMM model output is here, for each model there is a Date Folder and
# then OOS files
locIdx = 1  # '''WorkDrive'''

selection = os.listdir(inputDrive)[locIdx]

selectionLoc = os.path.join(inputDrive, selection)
# ''' location of WorkDrive'''
dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
DataLoc = os.path.join(hardDrivesLoc, selection, dataList[1])
path = 'MKL_Experiments'
MKLExpPath = os.path.join(DataLoc, path)
MKLSymbolPath = os.path.join(MKLExpPath, symbols[symbolIdx])

['BLT.L', 'CNA.L', 'PRU.L', 'BATS.L', 'CEY.L', 'APF.L', 'AZN.L', 'CCL.L', 'CPG.L', 'BARC.L', 'AV.L', 'AAL.L']


In [8]:
DataLoc

'/media/ak/WorkDrive/Data'

In [9]:
MKLSymbolPath

'/media/ak/WorkDrive/Data/MKL_Experiments/PRU.L'

In [10]:
SymbolCommonPaths =open_pickle_file(MKLSymbolPath,'LocDictsListCorrect.pkl' )

In [11]:
listCommonPaths = list(SymbolCommonPaths)

In [29]:
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return list(np.unique(x))

In [42]:
"".join((keyDate[1], "_Kernels.pkl", "rb"))

NameError: name 'keyDate' is not defined

In [37]:
a=listCommonPaths
np.unique([i[1] for i in a])

array(['20170118', '20170119', '20170120', '20170123', '20170124',
       '20170125', '20170126', '20170127', '20170130', '20170131',
       '20170301', '20170703', '20170704', '20170705', '20170706',
       '20170707', '20170710', '20170711', '20170712', '20170713',
       '20170714', '20170717', '20170718', '20170719', '20170720',
       '20170721', '20170724', '20170725', '20170726', '20170727',
       '20170728', '20170731', '20170801', '20170802', '20170803',
       '20170804', '20170807', '20170808', '20170809', '20170810',
       '20170811', '20170814', '20170815', '20170816', '20170817',
       '20170818', '20170821', '20170822', '20170823', '20170824',
       '20170825', '20170829', '20170830', '20170831', '20180201',
       '20180202', '20180205', '20180206', '20180207', '20180208',
       '20180209', '20180212', '20180213', '20180214', '20180215',
       '20180216', '20180219', '20180220', '20180221', '20180222',
       '20180223', '20180226', '20180227', '20180228', '201804

In [31]:
list(SymbolCommonPaths)

[('20170829', '20180411'),
 ('20170829', '20180228'),
 ('20170829', '20180227'),
 ('20170829', '20180202'),
 ('20170829', '20180214'),
 ('20170829', '20180405'),
 ('20170829', '20180205'),
 ('20170829', '20180416'),
 ('20170829', '20180412'),
 ('20170829', '20180419'),
 ('20170829', '20180219'),
 ('20170829', '20180413'),
 ('20170829', '20170830'),
 ('20170829', '20180226'),
 ('20170829', '20180221'),
 ('20170829', '20180417'),
 ('20170829', '20180207'),
 ('20170829', '20180215'),
 ('20170829', '20180220'),
 ('20170829', '20170831'),
 ('20170829', '20180222'),
 ('20170829', '20180209'),
 ('20170829', '20180208'),
 ('20170829', '20180410'),
 ('20170829', '20180213'),
 ('20170829', '20180201'),
 ('20170829', '20180420'),
 ('20170829', '20180418'),
 ('20170829', '20180409'),
 ('20170829', '20180403'),
 ('20170829', '20180206'),
 ('20170829', '20180216'),
 ('20170829', '20180404'),
 ('20170829', '20180212'),
 ('20170829', '20180406'),
 ('20170829', '20180223'),
 ('20170710', '20170801'),
 

In [29]:
if (Xtr.shape[0]==ytr.shape[0]):
    KLsimple = [HPK_kernel(Xtr,degree=d) for d in range(1,11)]
    KLrbf = [RBF(Xtr, gamma=gamma) for gamma in [1., 10, 100.]] 
    print ('training EasyMKL...for polynomials and RBF', end='')
    clfEasy = EasyMKL(lam=0.1).fit(KLsimple,ytr)#combining kernels with the EasyMKL algorithm
    clfRBF = EasyMKL(lam=0.1).fit(KLrbf,ytr)  
    print('------')
    print('finished training')
else:
    print('Shapes dont match.')
    pass


training EasyMKL...for polynomials and RBF------
finished training


In [45]:
os.listdir(MKLExpPath)
"/".join((MKLSymbolPath,'Kernels'))
# os.listdir(MKLSymbolPath)

'/media/ak/WorkDrive/Data/MKL_Experiments/PRU.L/Kernels'

In [ ]:


columns = ['feature{}'.format(ii) for ii in range(1, 5)] 

df = pd.DataFrame({col: np.random.randn(10) for col in columns})

path = '/home/ak/Documents/Research/temp/mkl_example.dmp' 

df['labels'] = np.random.choice([0, 1], size=len(df))
dump_svmlight_file(df[columns].values, df['labels'].values, path) 

X,Y = load_svmlight_file(path)
X = X.toarray()	#Important! MKLpy require dense matrices!
X = rescale_01(X)
X = normalization(X) 

KL = [HPK_kernel(X,degree=d) for d in range(1,11)]
KL2 = [RBF(X, gamma=gamma) for gamma in [1., 10, 100.]] 

EasyMKL().fit(KL2,Y)






In [ ]:
df['labels']

if __name__ == '__main__':
    xx, yy = np.meshgrid(np.linspace(-5, 5, 40), np.linspace(-5, 5, 40))
    np.random.seed(0)
    X = np.random.randn(300, 2)
    y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)
    # y = X[:,0]>0

    y = np.array(y, dtype=np.int)
    y[y == 0] = -1

    # Define the kernels
    kernels = [RBF(10 ** k) for k in range(-5, 0)]  # some RBF kernels
    kernels.append(linear)  # Add linear kernel

    # fit the model
    clf = MultiKernelSVC(kernels=kernels, C=1e6, verbose=True, maxit=100,
                         tol=1e-5, p=1, store_objective=True)
    clf.fit(X, y)

    objective = clf.objective

    pl.close('all')
    pl.figure()
    pl.plot(objective)
    pl.xlabel('Iterations')
    pl.ylabel('Dual objective')

    # plot the line, the points, and the nearest vectors to the plane
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # pl.close('all')
    pl.figure()
    pl.set_cmap(pl.cm.Paired)
    pl.pcolormesh(xx, yy, Z)
    pl.scatter(X[:,0], X[:,1], c=y)

    pl.axis('tight')
    pl.show()

In [ ]:
y.shape

In [ ]:
# Where to save the figures
_pkl_path =('/home/ak/Documents/Data/features_test/')

#ticker='MS.N'
#symbols[3] #enter symbol you want - this will be made for various

_graphs_path= '/home/ak/Documents/Data/features_test/graphs/'
_models_path = '/home/ak/Documents/Data/features_test/models/'
_predictions_path ='/home/ak/Documents/Data/features_test/predictions/'
figures_path= ('/home/ak/Documents/Research/QFPaper/paper/figures/')

In [ ]:
##load data
ticker = 'ZLc1_3' #loads specific symbol
with open(_pkl_path + ticker+'_states_features_data.pickle', 'rb') as handle:
    ticker_features_ = pickle.load(handle)
with open(_pkl_path + ticker+'_states_labelled_data.pickle', 'rb') as handle:
    ticker_labelled_data_ = pickle.load(handle)

In [ ]:
def off_set(array_1, array_2):
    off_set= len(array_1)-len(array_2)
    return off_set

def simpleInputs(_X,_y):
    _offset =np.abs(off_set(_X,_y))
    if _offset==0:
        X = _X.as_matrix().astype(np.float)
        y=_y.astype(np.int)
    else:
        X= _X[:-_offset].as_matrix().astype(np.float)
        y = _y.astype(np.int)
    return X, y

In [ ]:
###the following bit is a bit redundant###
_data = ticker_labelled_data_  # data
_features = ticker_features_  # features
_keys = _data.keys()  # dates
###scalers###
mms = MinMaxScaler()
stdsc = StandardScaler()
_clfs=[]
_xr=len(_keys)-1 #what is this
T=1
M=_xr #what is this

In [ ]:
_keys

In [ ]:
for _idx in xrange(0, _xr): #replace this w _xr
    # various types of labels
    y_cc = _data[_keys[_idx]].iloc[:, 6].dropna().astype(int).values  # label_simple_ep_window_5_thresh_0.00-last 4 are NaN
    y_duration = _data[_keys[_idx]].iloc[:, 3].dropna().astype(int).values
    y_price = _data[_keys[_idx]].iloc[:, 1].dropna().astype(int).values # can be used for regression

    # feature set
    _gamma = _features[_keys[_idx]][2]
    _csi = _features[_keys[_idx]][3]
    _info = _features[_keys[_idx]][1]
    _fischer = _features[_keys[_idx]][0]
    short_rolling = _data[_keys[_idx]]['TradedPrice'].rolling(window=5).mean()
    long_rolling = _data[_keys[_idx]]['TradedPrice'].rolling(window=15).mean()
    ma_signal = (long_rolling - short_rolling).fillna(0)
    
    df_features = pd.concat([_csi, _gamma, _fischer,_data[_keys[_idx]]['TradedPrice']], axis=1)
    
    df_features['MA_signal'] = df_features['TradedPrice'].rolling(window=15).mean()-df_features['TradedPrice'].rolling(window=5).mean()
    df_features=df_features.dropna()
        
    
    
    y_ = y_cc
    X, y = simpleInputs(df_features, y_) #simple inputs just makes sure all the sizes fit
    X_p, y = simpleInputs(df_features['MA_signal'], y_)
           
    print("number of classes:",len(np.unique(y)))  # of classes
    print(len(X)==len(y))
    print(len(X_p)==len(y))


In [ ]:
y.shape

In [ ]:
if __name__ == '__main__':
    xx, yy = np.meshgrid(np.linspace(-5, 5, 40), np.linspace(-5, 5, 40))
    np.random.seed(0)
#     X = np.random.randn(300, 2)
#     y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)
#     # y = X[:,0]>0

#     y = np.array(y, dtype=np.int)
#     y[y == 0] = -1

    # Define the kernels
    kernels = [RBF(10 ** k) for k in range(-5, 0)]  # some RBF kernels
    kernels.append(linear)  # Add linear kernel

    # fit the model
    clf = MultiKernelSVC(kernels=kernels, C=1e6, verbose=True, maxit=100,
                         tol=1e-5, p=1, store_objective=True)
    clf.fit(X, y)

    objective = clf.objective

    pl.close('all')X = np.random.randn(300, 2)
#     y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)
#     # y = X[:,0]>0

#     y = np.array(y, dtype=np.int)
#     y[y == 0] = -1

    pl.figure()
    pl.plot(objective)
    pl.xlabel('Iterations')
    pl.ylabel('Dual objective')

    # plot the line, the points, and the nearest vectors to the plane
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # pl.close('all')
    pl.figure()
    pl.set_cmap(pl.cm.Paired)
    pl.pcolormesh(xx, yy, Z)
    pl.scatter(X[:,0], X[:,1], c=y)

    pl.axis('tight')
    pl.show()

In [ ]:
X.shape